In [1]:
# https://github.com/DataTalksClub/llm-zoomcamp/blob/main/03-vector-search/eval/evaluation-metrics.md

In [2]:
2+2

4

In [3]:
import pandas as pd
from tqdm.auto import tqdm
import json
import sys
import os
from dotenv import load_dotenv
# Load environment variables from .env
load_dotenv("/home/jovyan/.envrc")

True

In [4]:
from sentence_transformers import SentenceTransformer

In [5]:
model_name = 'multi-qa-MiniLM-L6-cos-v1'
model = SentenceTransformer(model_name)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/11.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/opt/conda/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
from elasticsearch import Elasticsearch, ConnectionError

try:
    es = Elasticsearch(['http://elasticsearch:9200'])  # Use the service name
    if es.ping():
        print("Connected to Elasticsearch")
    else:
        print("Could not connect to Elasticsearch")
except ConnectionError as e:
    print(f"Connection error: {e}")


Connected to Elasticsearch


In [7]:
import json
import os

# Assuming final_results is defined as a list of dictionaries
# Example: final_results = [{'recommendationid': 1, 'question': 'What is the best game?'}, ...]
data_dir = os.path.abspath('../reviews-assistant/data/ground_truth')

# Define the output file path
output_file = os.path.join(data_dir, "ground_truth_retrieval.json")

# Load the JSON file content
with open(output_file, 'r', encoding='utf-8') as json_file:
    ground_truth = json.load(json_file)

In [8]:
ground_truth [-1]

{'document_id': 8862,
 'appid': '2208920',
 'review': {'appid': '2208920',
  'timestamp_query': 1727643597,
  'title': "Assassin's Creed Valhalla",
  'recommendationid': '174722275',
  'author.steamid': '76561199291372999',
  'author.playtimeforever': None,
  'author.playtime_last_two_weeks': 0,
  'author.playtime_at_review': 8996,
  'author.last_played': 1726115935,
  'language': 'english',
  'review': 'The Graphics are amazing. I also enjoyed the story line. I never expected Eivor to be the leader eventually. I thought he was just the caretaker.',
  'voted_up': True,
  'votes_up': 0,
  'timestamp_created': 1726050285,
  'timestamp_updated': 1726050285},
 'question': 'Does the character development meet expectations throughout the game?',
 'answer': "The character development, especially Eivor's role, is positively noted.",
 'section': 'characters'}

In [ ]:
from elasticsearch import Elasticsearch, NotFoundError, ConnectionError
from tqdm import tqdm
import json
import os

class ReviewIndexer:
    def __init__(self, es_host='http://elasticsearch:9200', index_name='steam-reviews', model=None):
        self.es = Elasticsearch([es_host])
        self.index_name = index_name
        self.model = model  # Expecting a SentenceTransformer model to encode text
        
        # Check the connection upon initialization
        self.check_connection()
        
        self.index_settings = {
            "settings": {
                "number_of_shards": 1,
                "number_of_replicas": 0
            },
            "mappings": {
                "properties": {
                    "appid": {"type": "keyword"},
                    "timestamp_query": {"type": "integer"},
                    "title": {"type": "keyword"},
                    "author.steamid": {"type": "keyword"},
                    "author.playtimeforever": {"type": "integer"},
                    "author.playtime_last_two_weeks": {"type": "integer"},
                    "author.playtime_at_review": {"type": "integer"},
                    "author.last_played": {"type": "integer"},
                    "language": {"type": "keyword"},
                    "review": {"type": "text"},
                    "voted_up": {"type": "boolean"},
                    "votes_up": {"type": "integer"},
                    "timestamp_created": {"type": "integer"},
                    "timestamp_updated": {"type": "integer"},
                    "question": {"type": "text"},
                    "answer": {"type": "text"},
                    "section": {"type": "keyword"},
                    "question_vector": {
                        "type": "dense_vector",
                        "dims": 384,
                        "index": True,
                        "similarity": "cosine"
                    },
                    "answer_vector": {
                        "type": "dense_vector",
                        "dims": 384,
                        "index": True,
                        "similarity": "cosine"
                    },
                    "question_answer_vector": {
                        "type": "dense_vector",
                        "dims": 384,
                        "index": True,
                        "similarity": "cosine"
                    },
                }
            }
        }

        # Drop the index if it exists and create a new one
        self.drop_and_create_index()

    def check_connection(self):
        """Check if Elasticsearch connection is established."""
        try:
            self.es.ping()
            print("Connected to Elasticsearch!")
        except ConnectionError:
            print("Failed to connect to Elasticsearch.")

    def drop_and_create_index(self):
        """Delete the existing index if it exists and create a new one."""
        try:
            if self.es.indices.exists(index=self.index_name):
                self.es.indices.delete(index=self.index_name)
                print(f"Index '{self.index_name}' deleted.")
            self.es.indices.create(index=self.index_name, body=self.index_settings)
            print(f"Index '{self.index_name}' created.")
        except Exception as e:
            print(f"Error creating index: {e}")

    def encode_vectors(self, question, answer):
        """Generate vectors for question, answer, and a combined question + answer."""
        question_vector = self.model.encode(question) if question else [0]*384
        answer_vector = self.model.encode(answer) if answer else [0]*384
        combined_text = f"{question} {answer}" if question and answer else ""
        question_answer_vector = self.model.encode(combined_text) if combined_text else [0]*384
        return question_vector, answer_vector, question_answer_vector

    def prepare_document(self, review):
        """Prepare the document to be indexed."""
        question_vector, answer_vector, question_answer_vector = self.encode_vectors(review["question"], review["answer"])
        
        return {
            "appid": review["appid"],
            "timestamp_query": review["review"]["timestamp_query"],
            "title": review["review"]["title"],
            "author.steamid": review["review"]["author.steamid"],
            "author.playtimeforever": review["review"]["author.playtimeforever"],
            "author.playtime_last_two_weeks": review["review"]["author.playtime_last_two_weeks"],
            "author.playtime_at_review": review["review"]["author.playtime_at_review"],
            "author.last_played": review["review"]["author.last_played"],
            "language": review["review"]["language"],
            "review": review["review"]["review"],
            "voted_up": review["review"]["voted_up"],
            "votes_up": review["review"]["votes_up"],
            "timestamp_created": review["review"]["timestamp_created"],
            "timestamp_updated": review["review"]["timestamp_updated"],
            "question": review["question"],
            "answer": review["answer"],
            "section": review["section"],
            "question_vector": question_vector,
            "answer_vector": answer_vector,
            "question_answer_vector": question_answer_vector
        }

    def index_reviews(self, reviews):
        """Index the provided reviews into Elasticsearch."""
        for review in tqdm(reviews):
            # Prepare the document
            doc = self.prepare_document(review)

            # Index the document
            try:
                self.es.index(index=self.index_name, body=doc)
            except Exception as e:
                print(f"Error indexing document with appid {review['appid']}: {e}")

    def load_reviews_from_file(self, file_path):
        """Load reviews from a JSON file."""
        try:
            with open(file_path, 'r') as file:
                reviews = json.load(file)
                return reviews
        except Exception as e:
            print(f"Error loading reviews from file: {e}")
            return []

# Example usage
if __name__ == "__main__":
    from sentence_transformers import SentenceTransformer
    import os

    # Define the data directory and output file path
    data_dir = os.path.abspath('../reviews-assistant/data/ground_truth')
    output_file = os.path.join(data_dir, "ground_truth_retrieval.json")

    # Initialize the model
    model_name = 'multi-qa-MiniLM-L6-cos-v1'
    model = SentenceTransformer(model_name)

    # Load reviews from the specified JSON file
    indexer = ReviewIndexer(model=model)
    reviews = indexer.load_reviews_from_file(output_file)
    
    # Index the loaded reviews
    if reviews:
        indexer.index_reviews(reviews)
    else:
        print("No reviews found to index.")


/opt/conda/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Connected to Elasticsearch!
Index 'steam-reviews' created.


 41%|████      | 3654/8863 [03:36<05:02, 17.24it/s]

In [ ]:
# Code to check mappings in Elasticsearch
from elasticsearch import Elasticsearch

def check_index_mapping(es_host='http://elasticsearch:9200', index_name='steam-reviews'):
    es = Elasticsearch([es_host])

    try:
        # Get the index mappings
        response = es.indices.get_mapping(index=index_name)
        mapping = response[index_name]['mappings']
        
        # Check for vector fields and their dimensions
        vector_fields = ["question_vector", "answer_vector", "question_answer_vector"]
        
        for field in vector_fields:
            if field in mapping['properties']:
                dims = mapping['properties'][field]['dims']
                print(f"{field}: {dims} dimensions")
            else:
                print(f"{field} not found in index.")
    except Exception as e:
        print(f"Error retrieving index mapping: {e}")

# Run the check
check_index_mapping()


In [ ]:
from sentence_transformers import SentenceTransformer

def check_vector_dimensions(query, model_name='multi-qa-MiniLM-L6-cos-v1'):
    # Load the model
    model = SentenceTransformer(model_name)

    # Encode the query and get the vector
    vector = model.encode(query)

    # Check the dimensions of the vector
    vector_dimensions = len(vector)
    print(f"Generated vector has {vector_dimensions} dimensions.")

# Example Usage
query = 'Is God of War: Ragnarok a game for kids?'
check_vector_dimensions(query)

In [ ]:
from elasticsearch import Elasticsearch

def verify_appid_exists(es_host='http://elasticsearch:9200', index_name='steam-reviews', appid='2322010'):
    es = Elasticsearch([es_host])

    try:
        # Define a query to check if the appid exists
        search_query = {
            "query": {
                "term": {
                    "appid": appid  # Filter by appid
                }
            }
        }

        # Perform the search
        es_results = es.search(index=index_name, body=search_query)

        if es_results['hits']['total']['value'] > 0:
            print(f"AppID '{appid}' exists in the index.")
            for hit in es_results['hits']['hits']:
                print(hit['_id'])
        else:
            print(f"AppID '{appid}' not found in the index.")

    except Exception as e:
        print(f"Error checking for AppID: {e}")

# Run the check for appid 2322010 (God of War: Ragnarok)
# verify_appid_exists(appid="552520")
verify_appid_exists(appid="2322010")

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from elasticsearch import Elasticsearch

class ReviewReader:
    def __init__(self, es_host='http://elasticsearch:9200', index_name='steam-reviews', model=None):
        self.es = Elasticsearch([es_host])
        self.index_name = index_name
        self.model = model  # Ensure the model is passed for generating embeddings

    def read_all_reviews(self):
        """Retrieve all reviews from the index."""
        try:
            response = self.es.search(index=self.index_name, body={"query": {"match_all": {}}})
            return response['hits']['hits']  # returns the list of documents
        except Exception as e:
            print(f"Error retrieving documents: {e}")
            return []

    def read_review_by_appid(self, appid):
        """Retrieve a specific review by appid."""
        try:
            response = self.es.search(index=self.index_name, body={
                "query": {
                    "term": {
                        "appid": appid  # Search by appid
                    }
                }
            })
            return response['hits']['hits']  # returns the list of documents matching the appid
        except Exception as e:
            print(f"Error retrieving document with appid {appid}: {e}")
            return []

    def read_reviews_knn(self, query, title, vector_field="answer_vector", num_results=5):
    # def read_reviews_knn(self, query, vector_field="answer_vector", num_results=5):
        """Retrieve reviews using KNN search only to test vector querying."""
        try:
            # Ensure the model is set
            if self.model is None:
                raise ValueError("Model for embedding generation is not initialized.")

            # Generate the embedding vector from the query
            # knn_vector = self.model.encode(query).tolist()

            # Define the KNN query
            knn_query = {
                "field": vector_field,  # Use the specified vector field (e.g., "answer_vector")
                "query_vector": knn_vector,
                "k": num_results,
                "num_candidates": 10000,  # Adjust as necessary
                "filter": {
                    "term": {
                        "title": title  # Filter by title
                    }
                }
            }

            # Perform the search
            es_results = self.es.search(index=self.index_name, body={"knn": knn_query})

            result_docs = []

            # Collect results
            for hit in es_results['hits']['hits']:
                result_docs.append(hit['_source'])  # Append only the source of each document

            if not result_docs:
                print("No results found.")
            return result_docs

        except Exception as e:
            print(f"Error executing KNN search: {e}")
            return []
    

    def read_reviews_knn_and_keyword(self, field, query, vector, title, num_results=5):
    # def read_reviews_knn_and_keyword(self, field, query, vector, num_results=5):
        """Retrieve reviews using both KNN and keyword search in a single query."""
        try:
            # Ensure the model is set
            if self.model is None:
                raise ValueError("Model for embedding generation is not initialized.")

            # Define the KNN query
            knn_query = {
                "field": field,  # Use the specified vector field (e.g., "answer_vector")
                "query_vector": vector,
                "k": num_results,
                "num_candidates": 10000,  # Adjust as necessary
                "boost": 0.5,
                "filter": {
                    "term": {
                        "title": title  # Filter by title
                    }
                }
            }

            # Define the keyword search query
            keyword_query = {
                "bool": {
                    "must": {
                        "multi_match": {
                            "query": query,
                            "fields": ["question^3", "answer", "section"],  # Relevant fields for keyword search
                            "type": "best_fields",
                            "boost": 0.5
                        }
                    },
                    "filter": {
                        "term": {
                            "title": title  # Filter by title
                        }
                    }
                }
            }

            # Combine the KNN and keyword search queries
            search_query = {
                "knn": knn_query,
                "query": keyword_query,
                "size": num_results,  # Limit the number of results
                "_source": ["answer", "section", "question", "title"]
            }

            # Perform the search
            es_results = self.es.search(index=self.index_name, body=search_query)
            result_docs = []

            # Collect results
            for hit in es_results['hits']['hits']:
                result_docs.append(hit['_source'])  # Append only the source of each document

            return result_docs

        except Exception as e:
            print(f"Error executing KNN and keyword search: {e}")
            return []


In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from elasticsearch import Elasticsearch

class ReviewReader:
    def __init__(self, es_host='http://elasticsearch:9200', index_name='steam-reviews', model=None):
        self.es = Elasticsearch([es_host])
        self.index_name = index_name
        self.model = model  # Ensure the model is passed for generating embeddings

    def read_all_reviews(self):
        """Retrieve all reviews from the index."""
        try:
            response = self.es.search(index=self.index_name, body={"query": {"match_all": {}}})
            return response['hits']['hits']  # returns the list of documents
        except Exception as e:
            print(f"Error retrieving documents: {e}")
            return []

    def read_review_by_appid(self, appid):
        """Retrieve a specific review by appid."""
        try:
            response = self.es.search(index=self.index_name, body={
                "query": {
                    "term": {
                        "appid": appid  # Search by appid
                    }
                }
            })
            return response['hits']['hits']  # returns the list of documents matching the appid
        except Exception as e:
            print(f"Error retrieving document with appid {appid}: {e}")
            return []

    def read_reviews_knn(self, query, title, vector_field="answer_vector", num_results=5):
    # def read_reviews_knn(self, query, vector_field="answer_vector", num_results=5):
        """Retrieve reviews using KNN search only to test vector querying."""
        try:
            # Ensure the model is set
            if self.model is None:
                raise ValueError("Model for embedding generation is not initialized.")

            # Generate the embedding vector from the query
            # knn_vector = self.model.encode(query).tolist()

            # Define the KNN query
            knn_query = {
                "field": vector_field,  # Use the specified vector field (e.g., "answer_vector")
                "query_vector": knn_vector,
                "k": num_results,
                "num_candidates": 10000,  # Adjust as necessary
                "filter": {
                    "term": {
                        "title": title  # Filter by title
                    }
                }
            }

            # Perform the search
            es_results = self.es.search(index=self.index_name, body={"knn": knn_query})

            result_docs = []

            # Collect results
            for hit in es_results['hits']['hits']:
                result_docs.append(hit['_source'])  # Append only the source of each document

            if not result_docs:
                print("No results found.")
            return result_docs

        except Exception as e:
            print(f"Error executing KNN search: {e}")
            return []
    

    def read_reviews_knn_and_keyword(self, field, query, vector, title, num_results=5):
    # def read_reviews_knn_and_keyword(self, field, query, vector, num_results=5):
        """Retrieve reviews using both KNN and keyword search in a single query."""
        try:
            # Ensure the model is set
            if self.model is None:
                raise ValueError("Model for embedding generation is not initialized.")

            # Define the KNN query
            knn_query = {
                "field": field,  # Use the specified vector field (e.g., "answer_vector")
                "query_vector": vector,
                "k": num_results,
                "num_candidates": 10000,  # Adjust as necessary
                "boost": 0.5,
                "filter": {
                    "term": {
                        "title": title  # Filter by title
                    }
                }
            }

            # Define the keyword search query
            keyword_query = {
                "bool": {
                    "must": {
                        "multi_match": {
                            "query": query,
                            "fields": ["question^3", "answer", "section"],  # Relevant fields for keyword search
                            "type": "best_fields",
                            "boost": 0.5
                        }
                    },
                    "filter": {
                        "term": {
                            "title": title  # Filter by title
                        }
                    }
                }
            }

            # Combine the KNN and keyword search queries
            search_query = {
                "knn": knn_query,
                "query": keyword_query,
                "size": num_results,  # Limit the number of results
                "_source": ["answer", "section", "question", "title"]
            }

            # Perform the search
            es_results = self.es.search(index=self.index_name, body=search_query)
            result_docs = []

            # Collect results
            for hit in es_results['hits']['hits']:
                result_docs.append(hit['_source'])  # Append only the source of each document

            return result_docs

        

        except Exception as e:
            print(f"Error executing KNN and keyword search: {e}")
            return []

    
    def compute_rrf(self, rank, k=60):
        """Compute Reciprocal Rank Fusion (RRF) score."""
        return 1 / (k + rank)

    
    def read_reviews_knn_and_keyword_rrf(self, field, query, vector, title, k=60, num_results=5):
        """Retrieve reviews using both KNN and keyword search, applying RRF for fusion."""
        try:
            # Ensure the model is set
            if self.model is None:
                raise ValueError("Model for embedding generation is not initialized.")

            # Define the KNN query
            knn_query = {
                "field": field,  # Use the specified vector field (e.g., "answer_vector")
                "query_vector": vector,
                "k": num_results,
                "num_candidates": 10000,
                "boost": 0.5,
                "filter": {
                    "term": {
                        "title": title  # Filter by title
                    }
                }
            }

            # Define the keyword search query
            keyword_query = {
                "bool": {
                    "must": {
                        "multi_match": {
                            "query": query,
                            "fields": ["question", "answer", "section"],
                            "type": "best_fields",
                            "boost": 0.5
                        }
                    },
                    "filter": {
                        "term": {
                            "title": title  # Filter by title
                        }
                    }
                }
            }

            # Execute the KNN search
            knn_results = self.es.search(index=self.index_name, body={"knn": knn_query, "size": num_results})['hits']['hits']

            # Execute the keyword search
            keyword_results = self.es.search(index=self.index_name, body={"query": keyword_query, "size": num_results})['hits']['hits']

            # Initialize the RRF score dictionary
            rrf_scores = {}

            # Calculate RRF for KNN results
            for rank, hit in enumerate(knn_results):
                doc_id = hit['_id']
                rrf_scores[doc_id] = self.compute_rrf(rank + 1, k)
                print(f"KNN: Doc {doc_id} rank {rank + 1}, RRF score: {rrf_scores[doc_id]}")

            # Calculate RRF for Keyword results
            for rank, hit in enumerate(keyword_results):
                doc_id = hit['_id']
                if doc_id in rrf_scores:
                    rrf_scores[doc_id] += self.compute_rrf(rank + 1, k)
                else:
                    rrf_scores[doc_id] = self.compute_rrf(rank + 1, k)
                print(f"Keyword: Doc {doc_id} rank {rank + 1}, RRF score: {rrf_scores[doc_id]}")

            # Sort documents based on the RRF scores
            reranked_docs = sorted(rrf_scores.items(), key=lambda x: x[1], reverse=True)

            # Get the top K documents by RRF scores
            final_results = []
            for doc_id, score in reranked_docs[:num_results]:
                doc = self.es.get(index=self.index_name, id=doc_id)
                final_results.append(doc['_source'])

            return final_results

        except Exception as e:
            print(f"Error executing KNN and keyword search with RRF: {e}")
            return []


In [ ]:
# Example Usage

# Initialize the SentenceTransformer model for embedding generation
model_name = 'multi-qa-MiniLM-L6-cos-v1'
model = SentenceTransformer(model_name)

# Create a reader instance with the model
reader = ReviewReader(model=model)

In [ ]:
# Example of reading all indexed reviews
all_reviews = reader.read_all_reviews()
print("All Indexed Reviews:")
for review in all_reviews:
    print(review['_id'])  # Print the id of each review
    # print(review['_source'])  # Print the source of each review

In [ ]:
# Example of reading specific review by appid
specific_reviews = reader.read_review_by_appid("2322010")
print("\nSpecific Reviews for appid 2322010  :")
for review in specific_reviews:
    print("-" * 79)
    print(review['_id'])  # Print the source of the specific review
    print(review['_source'])  # Print the source of the specific review

In [ ]:
ground_truth = [
    {
        'title': 'God of War: Ragnarok',
        'question': "Is this game for kids?"
    },
    {
        'title': 'Far Cry 5',
        'question': "Is this game open world with outpost capture mechanics?"
    },
    # Add more ground truth examples as needed
]

In [ ]:
# def hit_rate(relevance_total):
#     cnt = 0
#     print("cnt:", cnt)

#     for line in relevance_total:
#         if True in line:
#             cnt = cnt + 1

#     print("cnt:", cnt)

#     return cnt / len(relevance_total)

In [ ]:
def hit_rate(relevance_total):
    # Count the total number of True values
    total_true_count = sum(sum(line) for line in relevance_total)

    # Calculate the hit rate (percentage of True values over total values)
    total_values = sum(len(line) for line in relevance_total)  # total number of elements
    hit_rate_value = total_true_count / total_values if total_values > 0 else 0.0

    print("Total count of True values:", total_true_count)
    print("Hit Rate:", hit_rate_value)

    return hit_rate_value


In [ ]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [ ]:
def mrr(relevance_total):
    total_score = 0.0
    total_documents = 0  # To count all documents
    num_lines = len(relevance_total)

    # Calculate the total score by summing the reciprocal ranks of True values
    for line_index, line in enumerate(relevance_total):
        line_score = 0.0  # Score for the current line
        for rank, is_relevant in enumerate(line):
            total_documents += 1  # Count each document
            if is_relevant:
                line_score += 1 / (rank + 1)  # Add the reciprocal rank
                print(f"Line {line_index + 1}: Found relevant at rank {rank + 1}, score added: {1 / (rank + 1)}")
            else:
                print(f"Line {line_index + 1}: Found irrelevant at rank {rank + 1}")

        # If there's a relevant item in the line, accumulate the score
        if line_score > 0:
            total_score += line_score
            print(f"Line {line_index + 1}: Total line score: {line_score}")

    # Calculate the mean reciprocal rank using total_documents
    mean_reciprocal_rank = total_score / total_documents if total_documents > 0 else 0.0
    print(f"Total Score: {total_score}, Total Documents: {total_documents}, Mean Reciprocal Rank: {mean_reciprocal_rank}")

    return mean_reciprocal_rank


In [ ]:
field='question_answer_vector'
question="Is God of War launching with controller?"
title="God of War: Ragnarok"
# title="Far Cry 5"

In [ ]:
v_q = model.encode(question)

In [ ]:
response = reader.read_reviews_knn_and_keyword(field=field, query=question, vector=v_q, title=title, num_results=5)
# response = reader.read_reviews_knn_and_keyword(field=field, query=question, vector=v_q, num_results=5)

In [ ]:
response

In [ ]:
def question_text_hybrid(q):
    field='question_answer_vector'
    # question="Is God of War launching with proper controller support?"
    # title="God of War: Ragnarok"
    question = q['question']
    title = q['title']
    
    v_q = model.encode(question)

    return reader.read_reviews_knn_and_keyword(field=field, query=question, title=title, vector=v_q, num_results=10)
    # return reader.read_reviews_knn_and_keyword(field=field, query=question, vector=v_q, num_results=5)

In [ ]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        print("q:\n", q)
        
        doc_id = q['title']
        print("doc_id:\n", doc_id)
        
        results = search_function(q)
        print("results:\n", results[0]["answer"])
        
        relevance = [d['title'] == doc_id for d in results]
        print("relevance:\n", relevance)
        
        relevance_total.append(relevance)
        print("relevance_total:\n", relevance_total)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [ ]:
ground_truth_sample = [
    {
        "title": "God of War: Ragnarok",
        "question": "Is the game worth the price if it's performance in considered?"
    },
    # Add more ground truth examples as needed
]

In [ ]:
evaluate(ground_truth=ground_truth_sample, search_function=question_text_hybrid)

In [ ]:
def question_text_hybrid_rrf(q):
    field='question_answer_vector'
    question = q['question']
    title = q['title']
    
    v_q = model.encode(question)

    return reader.read_reviews_knn_and_keyword_rrf(field=field, query=question, title=title, vector=v_q, num_results=10)
    # return reader.read_reviews_knn_and_keyword(field=field, query=question, vector=v_q, num_results=5)

In [ ]:
evaluate(ground_truth=ground_truth_sample, search_function=question_text_hybrid_rrf)

In [ ]:
from openai import OpenAI

client = OpenAI(
  api_key=os.environ['OPENAI_API_KEY'],  # this is also the default, it can be omitted
)

In [ ]:
def build_prompt(query, search_results):
    answer_llm = ""

    if 'entry_template' in globals():  # Check if entry_template exists in global scope
        for doc in search_results:
            answer_llm += entry_template.format(**doc) + "\n\n"
    else:
        for doc in search_results:
            # Fallback formatting if entry_template is missing
            answer_llm += str(doc) + "\n\n"

    prompt = prompt_template.format(question=query, answer_llm=answer_llm).strip()
    return prompt

In [ ]:
def llm(prompt, model='gpt-4o-mini'):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [ ]:
def rag(query, model='gpt-4o-mini'):
    # search_results = search(query)
    search_results = reader.read_reviews_knn_and_keyword(field=field, query=question, title=title, vector=v_q, num_results=5)
    print(search_results)
    prompt = build_prompt(query, search_results)
    #print(prompt)
    answer = llm(prompt, model=model)
    # print(answer)
    return answer

In [ ]:
prompt_template = """
You are an expert evaluator for a RAG system.
Your task is to analyze the relevance of the generated answer to the given question.
Based on the relevance of the generated answer, you will classify it
as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

Here is the data for evaluation:

Question: {question}
Generated Answer: {answer_llm}

Please analyze the content and context of the generated answer in relation to the question
and provide your evaluation in parsable JSON without using code blocks:

{{
  "relevance": "NON_RELEVANT" | "PARTLY_RELEVANT" | "RELEVANT",
  "explanation": "[Provide a brief explanation for your evaluation]"
}}
""".strip()

In [ ]:
ground_truth[0]

In [ ]:
def build_prompt(query, search_results):
    answer_llm = ""

    if 'entry_template' in globals():  # Check if entry_template exists in global scope
        for doc in search_results:
            answer_llm += entry_template.format(**doc) + "\n\n"
    else:
        for doc in search_results:
            # Fallback formatting if entry_template is missing
            answer_llm += str(doc) + "\n\n"

    prompt = prompt_template.format(question=query, answer_llm=answer_llm).strip()
    return prompt

def llm(prompt, model='gpt-4o-mini'):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

def rag(query, model='gpt-4o-mini'):
    # search_results = search(query)
    search_results = reader.read_reviews_knn_and_keyword(field=field, query=query, title=title, vector=v_q, num_results=5)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt, model=model)
    
    # Return both the query and the answer
    return query, answer

import json
from tqdm import tqdm

evaluations = []

for record in tqdm(ground_truth):
    question = record['question']
    
    # Get LLM answer and the question
    query, answer_llm = rag(question)

    # Format the prompt with the question and LLM answer
    prompt = prompt_template.format(
        question=query,
        answer_llm=answer_llm
    )

    # Get evaluation from LLM
    evaluation = llm(prompt)
    evaluation = json.loads(evaluation)

    # Append a dictionary with record and evaluation details
    evaluations.append({
        "record": record,
        "evaluation": {
            "query": query,
            "answer_llm": answer_llm,
            "evaluation": evaluation
        }
    })

In [ ]:
evaluations[-1]

In [ ]:
evaluations[0]['record']['document_id']

In [ ]:
# Directory to save the JSON file
data_dir = os.path.abspath('../reviews-assistant/data/ground_truth')

# Define the output file path
output_file = os.path.join(data_dir, "ground_truth_evaluation.json")

# Save evaluations as JSON
with open(output_file, 'w') as file:
    json.dump(evaluations, file, indent=4)

print(f"Data has been saved to {output_file}")

In [ ]:
import pandas as pd
import json

# Assume `evaluations` is a list of dictionaries containing the evaluation data
# Example: evaluations = [ {...}, {...}, ..., {...} ]
# Load your evaluations JSON from file or it might already be defined in your script.
# If it's in a JSON file, you can load it as follows:
# with open('path_to_your_evaluations.json', 'r') as file:
#     evaluations = json.load(file)

# Assuming evaluations is already available as a list of dictionaries:
# Initialize an empty list to hold the records for the DataFrame
records = []

# Iterate through each evaluation entry
for evaluation in evaluations:
    record = evaluation['record']  # Extract the record part
    eval_info = evaluation['evaluation']  # Extract the evaluation part

    # Append a dictionary with the necessary data
    records.append({
        "document_id": record['document_id'],
        "appid": record['appid'],
        "title": record['review']['title'],
        "review": record['review']['review'],
        "voted_up": record['review']['voted_up'],
        "votes_up": record['review']['votes_up'],
        "timestamp_query": record['review']['timestamp_query'],
        "question": record['question'],
        "answer": record['answer'],
        "section": record['section'],
        "evaluation_query": eval_info['query'],
        # "evaluation_answer_llm": json.loads(eval_info['answer_llm']),  # Parse the answer_llm JSON string
        "evaluation_relevance": eval_info['evaluation']['relevance'],
        "evaluation_explanation": eval_info['evaluation']['explanation'],
    })

# Create the DataFrame
df = pd.DataFrame(records)

# Display the DataFrame
# print(df.head())  # Print the first few rows of the DataFrame for verification


In [ ]:
df

In [ ]:
# df_eval = pd.DataFrame(evaluations, columns=['question', 'record', 'answer', 'evaluation'])

# df_eval['appid'] = df_eval.record.apply(lambda d: d['appid'])
# df_eval['question'] = df_eval.record.apply(lambda d: d['question'])

# df_eval['relevance'] = df_eval.evaluation.apply(lambda d: d['relevance'])
# df_eval['explanation'] = df_eval.evaluation.apply(lambda d: d['explanation'])

# del df_eval['record']
# del df_eval['evaluation']

In [ ]:
df.evaluation_relevance.value_counts()

In [ ]:
df.evaluation_relevance.value_counts(normalize=True)

In [ ]:
df[df["evaluation_relevance"]=="RELEVANT"]

In [ ]:
df[df["evaluation_relevance"]=="NON_RELEVANT"]

In [ ]:
df[df["appid"]=="2322010"]